In [1]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv
import time # API 서버 부하를 줄이기 위해 time 라이브러리 추가

# --- dotenv 설정 ---
load_dotenv()
api_key = os.getenv("kangwon_business_area") 

if not api_key:
    print("❌ 오류: .env 파일에서 'kangwon_business_area' 키를 찾을 수 없습니다.")
else:
    url = "https://dataapi.wehago.com/data/"
    headers = {
        "apicode": api_key
    }
    
    # --- 반복문 로직 ---
    all_data_list = []  # 모든 데이터를 저장할 빈 리스트 (장바구니)
    start_index = 0
    chunk_size = 500    # 한 번에 요청할 개수 (0~499는 500개)

    print("데이터 수집을 시작합니다...")

    while True:
        # 1. 'from'과 'to' 파라미터를 동적으로 설정
        params = {
            "dataset_key": "DATASET_SUNDO_11",
            "from": start_index,
            "to": start_index + chunk_size - 1
        }
        
        print(f"  -> {start_index}부터 {start_index + chunk_size - 1}까지 요청 중...")

        try:
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                
                # 2. 'data' 키에서 실제 데이터 리스트를 가져옴
                data_chunk = data.get('data') # .get()을 사용하면 'data' 키가 없어도 오류 안남

                # 3. 데이터가 비어있는지 확인
                if not data_chunk:
                    # 서버가 빈 리스트([])를 반환하면, 더 이상 데이터가 없다는 의미
                    print("✅ 모든 데이터를 수집했습니다. 반복을 종료합니다.")
                    break
                
                # 4. (중요) 가져온 데이터를 '장바구니' 리스트에 추가
                all_data_list.extend(data_chunk)
                
                # 5. 다음 루프를 위해 시작 인덱스 업데이트
                start_index += chunk_size
                
                # (권장) API 서버에 대한 예의로, 요청 사이에 짧은 휴식 추가
                time.sleep(0.1) # 0.1초 대기

            else:
                # API 호출 자체가 실패한 경우 (400, 404, 500 등)
                print(f"❌ 오류 발생 (Status Code: {response.status_code})")
                print(response.text)
                break # 오류 발생 시 반복 중단

        except requests.exceptions.RequestException as e:
            print(f"연결 중 오류가 발생했습니다: {e}")
            break # 오류 발생 시 반복 중단

    # --- 반복문 종료 후 ---
    if all_data_list:
        # 6. '장바구니'에 모아둔 모든 데이터를 하나의 DataFrame으로 변환
        df_total = pd.DataFrame(all_data_list)
        
        print("\n--- 최종 DataFrame 변환 결과 ---")
        print(f"총 {len(df_total)}개의 행(row)을 가져왔습니다.")
        print(df_total.head())
    else:
        print("\n가져온 데이터가 없습니다.")

데이터 수집을 시작합니다...
  -> 0부터 499까지 요청 중...
  -> 500부터 999까지 요청 중...
  -> 1000부터 1499까지 요청 중...
  -> 1500부터 1999까지 요청 중...
  -> 2000부터 2499까지 요청 중...
  -> 2500부터 2999까지 요청 중...
  -> 3000부터 3499까지 요청 중...
  -> 3500부터 3999까지 요청 중...
  -> 4000부터 4499까지 요청 중...
  -> 4500부터 4999까지 요청 중...
  -> 5000부터 5499까지 요청 중...
  -> 5500부터 5999까지 요청 중...
  -> 6000부터 6499까지 요청 중...
  -> 6500부터 6999까지 요청 중...
  -> 7000부터 7499까지 요청 중...
  -> 7500부터 7999까지 요청 중...
  -> 8000부터 8499까지 요청 중...
  -> 8500부터 8999까지 요청 중...
  -> 9000부터 9499까지 요청 중...
  -> 9500부터 9999까지 요청 중...
  -> 10000부터 10499까지 요청 중...
  -> 10500부터 10999까지 요청 중...
  -> 11000부터 11499까지 요청 중...
  -> 11500부터 11999까지 요청 중...
  -> 12000부터 12499까지 요청 중...
  -> 12500부터 12999까지 요청 중...
  -> 13000부터 13499까지 요청 중...
  -> 13500부터 13999까지 요청 중...
  -> 14000부터 14499까지 요청 중...
  -> 14500부터 14999까지 요청 중...
  -> 15000부터 15499까지 요청 중...
  -> 15500부터 15999까지 요청 중...
  -> 16000부터 16499까지 요청 중...
  -> 16500부터 16999까지 요청 중...
  -> 17000부터 17499까지 요청 중...
  -> 17500부터 1

In [2]:
df_total.to_csv('강원도_사업체_공간집계정보.csv')

In [3]:
import pandas as pd
df=pd.read_csv('강원도_사업체_공간집계정보.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28443 entries, 0 to 28442
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      28443 non-null  int64  
 1   index           28443 non-null  int64  
 2   UPLFT_XCNTS     28443 non-null  float64
 3   UPLFT_YDNTS     28443 non-null  float64
 4   LOWRGT_XCNTS    28443 non-null  float64
 5   LOWRGT_YDNTS    28443 non-null  float64
 6   ID              28443 non-null  int64  
 7   BSNES_CNT       852 non-null    float64
 8   LABRR_CNT       852 non-null    float64
 9   LABRR_CNT_AVRG  852 non-null    float64
 10  SELNG_AMT       852 non-null    float64
 11  SELNG_AMT_AVRG  852 non-null    float64
 12  SPCE_INFO       28443 non-null  object 
dtypes: float64(9), int64(3), object(1)
memory usage: 2.8+ MB
